In [2]:
from collections import defaultdict
from typing import Dict, List, Any

class ConfigStore:
    def __init__(self):
        self.configs = []
        self.index = defaultdict(lambda: defaultdict(list))
        self.independent_vars = {}
        self.var_order = {}  # To keep track of the order of values for each variable

    def add_config(self, config: Dict[str, Any]):
        self.configs.append(config)
        for key, value in config.items():
            if key not in self.var_order:
                self.var_order[key] = []
            if value not in self.var_order[key]:
                self.var_order[key].append(value)
            if len(self.index[key][value]) == 0 or self.index[key][value][-1] != len(self.configs) - 1:
                self.index[key][value].append(len(self.configs) - 1)

    def add_independent_var(self, key: str, values: List[Any]):
        self.independent_vars[key] = values
        self.var_order[key] = values

    def query(self, constraints: Dict[str, List[Any]]) -> Dict[str, List[Any]]:
        if not constraints:
            return self._all_possible_values()

        valid_configs = set(range(len(self.configs)))
        for key, values in constraints.items():
            if key in self.independent_vars:
                continue
            key_valid = set()
            for value in values:
                key_valid.update(self.index[key][value])
            valid_configs.intersection_update(key_valid)

        result = defaultdict(set)
        for i in valid_configs:
            for key, value in self.configs[i].items():
                result[key].add(value)

        # Add independent variables
        for key, values in self.independent_vars.items():
            if key in constraints:
                result[key] = set(constraints[key]) & set(values)
            else:
                result[key] = set(values)

        # Convert sets to ordered lists based on original insertion order
        ordered_result = {}
        for key, values in result.items():
            ordered_result[key] = [v for v in self.var_order[key] if v in values]

        return ordered_result

    def _all_possible_values(self) -> Dict[str, List[Any]]:
        result = {key: self.var_order[key] for key in self.var_order if key not in self.independent_vars}
        result.update(self.independent_vars)
        return result

# Usage example
store = ConfigStore()

# Add configurations
configs = [
    {'a': 1, 'c': 3},
    {'a': 1, 'c': 5},
    {'a': 2, 'c': 5, 'd': 7},
    {'a': 2, 'c': 5, 'd': 8},
    {'a': 2, 'c': 6, 'd': 7},
    {'a': 2, 'c': 6, 'd': 8}
]
for config in configs:
    store.add_config(config)

# Add independent variable
store.add_independent_var('b', [3, 4])

# Query examples
print(store.query({'a': [1, 2], 'b': [3]}))
# Output: {'a': [1, 2], 'b': [3], 'c': [3, 5, 6], 'd': [7, 8]}

print(store.query({'a': [1]}))
# Output: {'a': [1], 'b': [3, 4], 'c': [3, 5]}

print(store.query({'a': [2], 'c': [5]}))
# Output: {'a': [2], 'b': [3, 4], 'c': [5], 'd': [7, 8]}

{'a': [1, 2], 'c': [3, 5, 6], 'd': [7, 8], 'b': [3]}
{'a': [1], 'c': [3, 5], 'b': [3, 4]}
{'a': [2], 'c': [5], 'd': [7, 8], 'b': [3, 4]}
